# ⚙️ Module 13: ADAS Implementation

This is the implementation phase. You will combine all the theory and individual components from the previous modules into one complete, functional Advanced Driver Assistance System: a Forward Collision Warning (FCW) system.

## Section 1: Your Mission

Your goal is to build a **complete Forward Collision Warning system** that:
1. Reads live motor speed (RPM) from the VESC.
2. Reads live object detections from the AI camera.
3. Estimates the distance to a detected 'person'.
4. Calculates the Time-to-Collision (TTC).
5. Issues multi-level audio alerts based on the TTC value.
6. Displays a real-time status dashboard.

## Section 2: Component Integration

First, let's bring all our functions and initializations into one place.

In [ ]:
# 2.1 - Full System Initialization
import sys, os, time
from gpiozero import Buzzer
from IPython.display import display, HTML, clear_output
sys.path.append(os.path.abspath('../'))
from student_api import VESCStudentAPI, AIStudentAPI

# -- Initialize Hardware --
buzzer = Buzzer(17)
ai_api = AIStudentAPI()
ai_api.start_camera()
vesc_api = VESCStudentAPI()
vesc = None
if vesc_api.start():
    controllers = []
    deadline = time.time() + 10.0
    while time.time() < deadline and not controllers:
        controllers = vesc_api.get_connected_controllers()
        if not controllers:
            time.sleep(0.5)

    if controllers:
        vesc = vesc_api.get_controller(controllers[0])
        print(f'✅ VESC & AI Systems Initialized.')
    else:
        print('❌ VESC: No controllers found within 10 seconds')
else:
    print('❌ VESC: Failed to start API')


In [ ]:
# 2.2 - Helper Functions from Module 12

WHEEL_CIRCUMFERENCE_M = 0.2
GEAR_RATIO = 12.0
CALIBRATION_DISTANCE_M = 2.0
CALIBRATION_BOX_HEIGHT_PX = 150

def estimate_distance(box_height_px):
    if box_height_px == 0:
        return float('inf')
    return (CALIBRATION_BOX_HEIGHT_PX / box_height_px) * CALIBRATION_DISTANCE_M

def get_speed_mps(rpm):
    revolutions_per_second = rpm / 60.0
    wheel_rps = revolutions_per_second / GEAR_RATIO
    return wheel_rps * WHEEL_CIRCUMFERENCE_M

def calculate_ttc(distance_m, speed_mps):
    if speed_mps <= 0.1:
        return float('inf')
    return distance_m / speed_mps

def ttc_to_brake_time(ttc_s):
    if ttc_s <= 1.0:
        return 3.0
    if ttc_s < 5.0:
        return max(3.0, min(10.0, 3.0 + (ttc_s - 1.0) * 1.75))
    return 10.0

def ttc_to_brake_current(ttc_s):
    if ttc_s <= 1.0:
        return 10.0
    if ttc_s < 5.0:
        return max(2.0, min(10.0, 10.0 - (ttc_s - 1.0) * 2.0))
    return 2.0

print("✅ Helper functions loaded.")


## Section 3: Complete FCW System

Now we combine everything into a single, continuous loop. Press the 'Interrupt' (square) button to stop the system.

In [ ]:
# --- FCW Configuration ---
CONFIDENCE_THRESHOLD = 0.7
TARGET_LABEL = 'person'
COOLDOWN = 3
last_alert_time = 0

TTC_CRITICAL = 1.0
TTC_WARNING = 3.0

try:
    print("✅ Forward Collision Warning System ACTIVE. Press Interrupt (■) to stop.")
    while True:
        rpm = vesc.get_rpm() if vesc else 0
        _, detections = ai_api.get_frame_and_detections()

        speed_mps = get_speed_mps(rpm)
        closest_person_dist = float('inf')

        for det in detections:
            if det['label'] == TARGET_LABEL and det['confidence'] > CONFIDENCE_THRESHOLD:
                box_height = det['box'][3]
                dist = estimate_distance(box_height)
                if dist < closest_person_dist:
                    closest_person_dist = dist

        ttc = calculate_ttc(closest_person_dist, speed_mps)
        brake_time_s = ttc_to_brake_time(ttc)
        brake_current_a = ttc_to_brake_current(ttc)

        status = "SAFE"
        alert_level = 0
        if ttc < TTC_CRITICAL:
            status = "CRITICAL"
            alert_level = 2
        elif ttc < TTC_WARNING:
            status = "WARNING"
            alert_level = 1

        if alert_level > 0 and (time.time() - last_alert_time > COOLDOWN):
            last_alert_time = time.time()
            if alert_level == 2:
                buzzer.beep(0.1, 0.1, 3)
                if vesc:
                    vesc.set_brake_current(brake_current_a, brake_time_s)
            elif alert_level == 1:
                buzzer.beep(0.2, 0.2, 1)

        clear_output(wait=True)
        status_color = 'lime' if status == 'SAFE' else ('orange' if status == 'WARNING' else 'red')
        html_out = f"""
        <div style="border: 2px solid #44B6E5; padding: 10px; font-family: 'Space Grotesk', monospace;">
             <p style="font-size: 24px;">FCW STATUS: <strong style="color:{status_color};">{status}</strong></p>
             <p><strong>TTC:</strong> {ttc:.2f} s</p>
             <p><strong>Distance:</strong> {closest_person_dist:.2f} m</p>
             <p><strong>Speed:</strong> {speed_mps * 3.6:.1f} km/h</p>
             <p><strong>Brake Plan:</strong> {brake_current_a:.1f}A over {brake_time_s:.1f}s</p>
        </div>
        """
        display(HTML(html_out))

        time.sleep(0.1)

except KeyboardInterrupt:
    print("\nFCW System stopped by user.")
finally:
    ai_api.stop_camera()
    if vesc_api.is_running(): vesc_api.stop()
    buzzer.close()


## Section 4: Testing

**SAFETY FIRST: Ensure the vehicle is on a secure test stand with the wheels OFF THE GROUND.**

### EXERCISE: System Validation
Follow this protocol and document your observations.

1. **Baseline (No Person):** Spin wheel by hand with no target.
   - *Expected:* SAFE, no buzzer, no brake sequence.
   - *Observed:*

2. **Warning Range:** Show a person target and hand-spin wheel slowly.
   - *Expected:* WARNING beep pattern may trigger, no strong intervention.
   - *Observed:*

3. **Critical Range:** Increase wheel speed with close target.
   - *Expected:* CRITICAL alert + TTC-based brake sequence where lower TTC leads to shorter ramp time.
   - *Observed:*


## Section 5: Knowledge Check

### EXERCISE: List 3 Improvements
Based on your testing, what are three ways you could make this FCW system more reliable or intelligent? (e.g., consider object size, false positives, etc.)

// 1. 
// 2. 
// 3. 

### EXERCISE: What's Next?
How could you improve the TTC-to-brake mapping to reduce false interventions while preserving fast response when TTC drops quickly?


// Your answer here

---
## Congratulations! You've completed Module 13 and built a real ADAS feature.